In [3]:
!pip install transformers
!pip install torch
!pip install numpy
!pip install pandas
!pip install datasets
!pip install scikit-learn
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached datasets-2.6.1-py3-none-any.whl (441 kB)
  Using cached xxhash-3.1.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (212 kB)
  Using cached multiprocess-0.70.14-py37-none-any.whl (115 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
  Using cached dill-0.3.5.1-py2.py3-none-any.whl (95 kB)
  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Using cached multiprocess-0.70.13-py37-none-any.whl (115 kB)
  Attempting uninstall: urllib3
    Found existing in

In [9]:
import os
import transformers
import pandas as pd
import pandas as pd
import numpy as np
import uuid
import random
import torch
from transformers import ElectraTokenizerFast, ElectraModel, Trainer, TrainingArguments, ElectraForTokenClassification
from transformers import pipeline
from transformers.convert_graph_to_onnx import convert
from transformers import DataCollatorForTokenClassification
from datasets.dataset_dict import Dataset, DatasetDict
from datasets import load_dataset, load_metric
from pathlib import Path
from sklearn.model_selection import train_test_split
import datasets
os.environ["WANDB_DISABLED"] = "true"
from datasets import load_dataset, load_metric

In [10]:
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

In [11]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "google/electra-small-discriminator"
batch_size = 8

In [12]:
model =  ElectraForTokenClassification.from_pretrained(model_checkpoint, num_labels=13)
tokenizer = ElectraTokenizerFast.from_pretrained(model_checkpoint, use_fast=True)

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForTokenClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.weight', 'c

In [16]:
!ls

sample_data  train.txt


In [17]:
f = open("train.txt", "r")
text = f.read()

In [18]:
updated_text = text.split("\n\t\n")

# New Section

# New Section

In [19]:
data = []
for sent in updated_text:
    sent_text =  []
    sent_tag = []
    for i in sent.split("\n"):
        w = i.split("\t")
        if w[-1] != "":
            sent_text.append(w[0])
            sent_tag.append(w[-1])
    data.append({"tokens":sent_text, "ner_tags":sent_tag,"id":uuid.uuid4().hex})

In [20]:
data_df = pd.DataFrame(data)
data_df.dropna()

,tokens,ner_tags,id
0,"[@paulwalk, It, 's, the, view, from, where, I,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-l...",97e1c448465b485ea7da909352c46c67
1,"[From, Green, Newsfeed, :, AHFA, extends, dead...","[O, O, O, O, B-group, O, O, O, O, O, O, O, O, ...",795e67eddddb488fb5bbcba1cb615ba1
2,"[Pxleyes, Top, 50, Photography, Contest, Pictu...","[B-corporation, O, O, O, O, O, O, O, O, O, O, O]",9c8169f1298546d4853b3ab6ffadebad
3,"[today, is, my, last, day, at, the, office, .]","[O, O, O, O, O, O, O, O, O]",ee5d5e4dac9a4426989949ff3e5264ad
4,"[4Dbling, 's, place, til, monday, ,, party, pa...","[B-person, O, O, O, O, O, O, O, O, O, O, O]",5b4f7c6e71ad4ef9b85cd76053810fcc
...,...,...,...
2390,"[Five, Bullpens, to, Stay, Away, From, in, MLB...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",76149cf40e2e47fea76376968bd477a7
2391,"[New, Dorky, Stuff, :, Friday, Review, Rundown...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",c484c56b39cc4088ba66fef8c4520472
2392,"[Oh, PEI, ,, i, always, forget, how, beautiful...","[O, B-person, O, O, O, O, O, O, O, O, O, O, O,...",38516533fb594387b335188326febefd
2393,"[workin, at, the, ol, Sunset, Bistro, tonight,...","[O, O, O, O, B-location, I-location, O, O, O, ...",a2b0e82d1c9f4a88a5b5a31309ead3ac


In [21]:
flat_list = [item for sublist in data_df['ner_tags'] for item in sublist]
m = set(flat_list)

In [22]:
# Creating the DataFrame from the dataset lists
train_data_df,val_data_df  = train_test_split(data_df,test_size=0.2, shuffle=True)

In [23]:
dataset_info = datasets.DatasetInfo(
    description="_DESCRIPTION",
    features=datasets.Features(
        {
            "id": datasets.Value("string"),
            "tokens": datasets.Sequence(datasets.Value("string")),
            "ner_tags": datasets.Sequence(
                datasets.features.ClassLabel(num_classes=13,
                    names=sorted(['O', 'B-corporation',
                                         'B-creative-work',
                                         'B-group',
                                         'B-location',
                                         'B-person',
                                         'B-product',
                                         'I-corporation',
                                         'I-creative-work',
                                         'I-group',
                                         'I-location',
                                         'I-person',
                                         'I-product'
                                       ])
                                                        )
                                                    )
                                                }
                                            ),
                                            supervised_keys=None,
                                            homepage="",
                                            citation="_CITATION",
                                        )

In [24]:
dataset_info

DatasetInfo(description='_DESCRIPTION', citation='_CITATION', homepage='', license='', features={'id': Value(dtype='string', id=None), 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(names=['B-corporation', 'B-creative-work', 'B-group', 'B-location', 'B-person', 'B-product', 'I-corporation', 'I-creative-work', 'I-group', 'I-location', 'I-person', 'I-product', 'O'], id=None), length=-1, id=None)}, post_processed=None, supervised_keys=None, task_templates=None, builder_name=None, config_name=None, version=None, splits=None, download_checksums=None, download_size=None, post_processing_size=None, dataset_size=None, size_in_bytes=None)

In [25]:
def map_df_rows(df):
    """
    Function to map DataFrame Label row with integers, The dictionary given in below
    function can be updated as per the requirments
    Args:
        df: Input DataFrame for which we need to transform ner_tags label
    Returns:
        List of mapped integers with labels
    """
    updated_rows = []
    # The following map dict can be updated if more annotations comes in
    map_df_rows_dict = {
                         "O":0,
                         "B-corporation":1,
                         "B-creative-work":2,
                         "B-group":3,
                         "B-location":4,
                         "B-person":5,
                         "B-product":6,
                         "I-corporation":7,
                         "I-creative-work":8,
                         "I-group":9,
                         "I-location":10,
                         "I-person":11,
                         "I-product":12}
    for row in list(df['ner_tags']):
        updated_rows.append(list(map(map_df_rows_dict.get, row)))

    return updated_rows


In [26]:
def tokenize_and_align_labels(examples):
    """
    Function for tokenizing and aligning labels
    """
    #tokenizer = self.artifacts.ner_model.get("tokenizer")
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True,max_length=512, is_split_into_words=True)
    print ("tokenized_inputs", tokenized_inputs.keys())
    labels = []
    task = "ner"
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        
        labels.append(label_ids)


    tokenized_inputs["labels"] = labels
    tokenized_inputs["tokens"] = examples["tokens"]
    tokenized_inputs["id"] = examples["id"]
    return tokenized_inputs

In [27]:
metric = load_metric("seqeval")
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


In [28]:
val_data_df.head()

,tokens,ner_tags,id
1540,"[@jeffstinco, or, @chuckcomeau, follow, me, fo...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",ccfe5bc2fb554982b8b1d06868b32869
1579,"[I, had, a, good, day, :), ,, how, y'all, doin...","[O, O, O, O, O, O, O, O, O, O, O]",14083152097847ecb1b0d5716636e744
829,"[Not, awful, ..., going, to, start, fresh, thi...","[O, O, O, O, O, O, O, O, O, O, O, O, O]",afa8bef331fc4a85b58d52f5fb28fb5e
441,"[RT, @askyfullofstars, :, AstronomersWithoutBo...","[O, O, O, B-group, O, O, O, O, O, O, O, O, O, ...",ad51c6fe70954b48863c326281aeabd2
1361,"[WHATS, GOODIE, EVERY1, COOLIN, COOLIN, FEELIN...","[O, O, O, O, O, O, O, O, O]",8a0eaec6c0104448a88800da52b1b298


In [29]:
train_data_df.head()

,tokens,ner_tags,id
1309,"[@CHRIS_Daughtry, Aw, ur, cute, when, u, get, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",e0a7745bd692468f9179064f6a1a9923
498,"[check, out, @140hours, auction, starts, tonig...","[O, O, O, O, O, O, O, O, O, O]",e094a8934c954e09854667fc89f45723
2195,"[@DesiiDanii4, nou, ze, is, al, een, week, bes...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",5dcae5d1797d438cabf3303bd175d82e
740,"[No, Good, Punk, :, Thug, knocks, 84-year-old,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",5a953a1360aa4e37a3e893ad98e32e7f
2077,"[@ahmong, I'm, thinking, they, trade, him, bef...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",c4b38db659694d809dd3493f4933071a


In [30]:
val_data_df['ner_tags'] = map_df_rows(val_data_df)
train_data_df['ner_tags'] = map_df_rows(train_data_df)

In [31]:
# Map the Labels with integers

train_data = Dataset.from_pandas(train_data_df,info= dataset_info, preserve_index=False)
val_data = Dataset.from_pandas(val_data_df,info= dataset_info, preserve_index=False)


In [32]:
dataset_dict = DatasetDict({"train": train_data, 'val': val_data})
label_list = dataset_dict["train"].features["ner_tags"].feature.names
label_list

['B-corporation',
 'B-creative-work',
 'B-group',
 'B-location',
 'B-person',
 'B-product',
 'I-corporation',
 'I-creative-work',
 'I-group',
 'I-location',
 'I-person',
 'I-product',
 'O']

In [33]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1916
    })
    val: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 479
    })
})

In [34]:
task = "ner"
label_all_tokens = True
# Tokenize the dataset context        
tokenized_datasets = dataset_dict.map(tokenize_and_align_labels, batched=True, remove_columns = dataset_dict["train"].column_names)
# Define Model


  0%|          | 0/2 [00:00<?, ?ba/s]

tokenized_inputs dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
tokenized_inputs dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


  0%|          | 0/1 [00:00<?, ?ba/s]

tokenized_inputs dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [35]:
tokenized_datasets.data

{'train': InMemoryTable
 id: string
 tokens: list<item: string>
   child 0, item: string
 input_ids: list<item: int32>
   child 0, item: int32
 token_type_ids: list<item: int8>
   child 0, item: int8
 attention_mask: list<item: int8>
   child 0, item: int8
 labels: list<item: int64>
   child 0, item: int64
 ----
 id: [["e0a7745bd692468f9179064f6a1a9923","e094a8934c954e09854667fc89f45723","5dcae5d1797d438cabf3303bd175d82e","5a953a1360aa4e37a3e893ad98e32e7f","c4b38db659694d809dd3493f4933071a","9aab433eee7141eb839a80c3f1f440d8","92d740eeed5a4eb8878bd8039b8974a9","940a0392a9e641398c6f05c0003b36f6","d88956fed481493d9f97bafd240d68fc","2256343096d04a718de3fcd0e5f51b10",...,"cf0e2957753a49cba5b698fb44e92740","20f5caa183954379b63f618952e55aa4","95df1f26e5484a4580bc5bbd217dd4db","94fbf73f76ae43508f81a8c572d2a941","0d4ffbcc0de4415c8abee37556225a2c","e3e8a3c103c749e598a3fe75de07ffbe","b48fb7ab588b495dbebba3b8f4ac0049","add237cc24f449d69f22a5122377c81a","362d435c726b4d44bdd2087a139b8521","420e70377

In [36]:
# Create Dataset Dictionary


#
data_collator = DataCollatorForTokenClassification(tokenizer)    
    

In [37]:
model_name = "electra_small"

In [38]:
training_args = TrainingArguments(f"{model_name}-finetuned-{task}",
              #  output_dir=tmp_dir,
                learning_rate=2e-5,
                per_device_train_batch_size=8,
                per_device_eval_batch_size=8,
                num_train_epochs=10,
                #logging_dir=os.path.join(tmp_dir, "logs"),
                logging_steps=max(1, len(train_data)//8//4),
                save_steps=max(1, len(train_data)//48//4),
                save_total_limit=2,              
                evaluation_strategy="steps",
                eval_steps=max(1, len(train_data)//8//4),
                seed=42
                
            )

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [39]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=tokenized_datasets["train"],           # training dataset
    eval_dataset=tokenized_datasets["val"],            # evaluation dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


In [40]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `ElectraForTokenClassification.forward` and have been ignored: tokens, id. If tokens, id are not expected by `ElectraForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1916
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2400
  Number of trainable parameters = 13486349
You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than 

Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
59,1.207100,0.379893,0.953661,0.971482,0.962489,0.955798
118,0.342500,0.301963,0.953661,0.971482,0.962489,0.955798
177,0.290400,0.288304,0.953661,0.971482,0.962489,0.955798
236,0.297900,0.283285,0.953661,0.971482,0.962489,0.955798
295,0.298300,0.270466,0.953661,0.971482,0.962489,0.955798
354,0.226100,0.233949,0.953661,0.971482,0.962489,0.955798
413,0.261300,0.215590,0.954103,0.971932,0.962935,0.957198
472,0.223800,0.213161,0.955282,0.973133,0.964125,0.958597
531,0.218300,0.212084,0.952262,0.970056,0.961077,0.956461
590,0.204500,0.200413,0.956157,0.973809,0.964902,0.960586


Saving model checkpoint to electra_small-finetuned-ner/checkpoint-9
Configuration saved in electra_small-finetuned-ner/checkpoint-9/config.json
Model weights saved in electra_small-finetuned-ner/checkpoint-9/pytorch_model.bin
tokenizer config file saved in electra_small-finetuned-ner/checkpoint-9/tokenizer_config.json
Special tokens file saved in electra_small-finetuned-ner/checkpoint-9/special_tokens_map.json
Saving model checkpoint to electra_small-finetuned-ner/checkpoint-18
Configuration saved in electra_small-finetuned-ner/checkpoint-18/config.json
Model weights saved in electra_small-finetuned-ner/checkpoint-18/pytorch_model.bin
tokenizer config file saved in electra_small-finetuned-ner/checkpoint-18/tokenizer_config.json
Special tokens file saved in electra_small-finetuned-ner/checkpoint-18/special_tokens_map.json
Saving model checkpoint to electra_small-finetuned-ner/checkpoint-27
Configuration saved in electra_small-finetuned-ner/checkpoint-27/config.json
Model weights saved i

TrainOutput(global_step=2400, training_loss=0.1931462961435318, metrics={'train_runtime': 330.7379, 'train_samples_per_second': 57.931, 'train_steps_per_second': 7.257, 'total_flos': 54137448445968.0, 'train_loss': 0.1931462961435318, 'epoch': 10.0})

In [41]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `ElectraForTokenClassification.forward` and have been ignored: tokens, id. If tokens, id are not expected by `ElectraForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 479
  Batch size = 8


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.17414917051792145,
 'eval_precision': 0.9614928030865113,
 'eval_recall': 0.9725328330206379,
 'eval_f1': 0.9669813080625302,
 'eval_accuracy': 0.9615441284809194,
 'eval_runtime': 1.1582,
 'eval_samples_per_second': 413.571,
 'eval_steps_per_second': 51.804,
 'epoch': 10.0}

In [42]:
trainer.save_model()

Saving model checkpoint to electra_small-finetuned-ner
Configuration saved in electra_small-finetuned-ner/config.json
Model weights saved in electra_small-finetuned-ner/pytorch_model.bin
tokenizer config file saved in electra_small-finetuned-ner/tokenizer_config.json
Special tokens file saved in electra_small-finetuned-ner/special_tokens_map.json


In [43]:
from google.colab import files
files.download('/content/electra_small-finetuned-ner/pytorch_model.bin') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
/content/electra_small-finetuned-ner/pytorch_model.bin